In [7]:
import cv2
import pytesseract
import numpy as np

# Initialize the webcam or video input
cap = cv2.VideoCapture(1)  # Webcam input
# cap = cv2.VideoCapture("file.mp4")  # Use video file if needed

if not cap.isOpened():
    cap = cv2.VideoCapture(0)  # Fallback to default camera if webcam is unavailable
if not cap.isOpened():
    raise IOError("Cannot open video")

# Function for image preprocessing
def preprocess_image(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Remove noise using Gaussian Blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding for better contrast
    threshold_img = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )

    return threshold_img

# Main loop for video capture and processing
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # Preprocess the frame for better OCR results
    preprocessed_frame = preprocess_image(frame)

    # Extract text from the preprocessed frame using pytesseract
    imgchar = pytesseract.image_to_string(preprocessed_frame)
    imgboxes = pytesseract.image_to_boxes(preprocessed_frame)

    # Display detected text on console
    print(f"Detected Text:\n{imgchar}")

    # Draw bounding boxes around detected text
    imgH, imgW, _ = frame.shape
    for b in imgboxes.splitlines():
        b = b.split(' ')
        x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
        cv2.rectangle(frame, (x, imgH - y), (w, imgH - h), (0, 255, 0), 2)

    # Display the original frame with bounding boxes
    cv2.imshow("OCR Result", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 